In [ ]:
import numpy as np
import xarray as xr

from eval_utilities import metrics
import matplotlib.pyplot as plt

In [ ]:
import yaml
with open(f"config.yaml") as stream:
    try:
        CONFIG = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
ds = xr.open_zarr(CONFIG["path_ec_euro"])

In [ ]:
fft_t2m = np.fft.fft(ds.data.sel(variable="t2m"), axis=0)
fft_gpp = np.fft.fft(ds.data.sel(variable="aco2gpp"), axis=0)
freq = np.fft.fftfreq(ds.sizes["time"], d=6*60*60)
mask = freq > 0

In [ ]:
fig, ax = plt.subplots()
fig.tight_layout()

# Day:
ax.axvline(1/(24*60*60), color="tab:grey", ls="dashed")
ax.text(1/(24*60*60), 0.99, 'day', color='tab:grey', ha='right', va='top', rotation=90, transform=ax.get_xaxis_transform())

# Month:
ax.axvline(1/(30*24*60*60), color="tab:grey", ls="dashed")
ax.text(1/(30*24*60*60), 0.99, 'month', color='tab:grey', ha='right', va='top', rotation=90, transform=ax.get_xaxis_transform())

# Season:
ax.axvline(4/(365*24*60*60), color="tab:grey", ls="dashed")
ax.text(4/(365*24*60*60), 0.99, 'season', color='tab:grey', ha='right', va='top', rotation=90, transform=ax.get_xaxis_transform())

# Year:
ax.axvline(1/(365*24*60*60), color="tab:grey", ls="dashed")
ax.text(1/(365*24*60*60), 0.99, 'year', color='tab:grey', ha='right', va='top', rotation=90, transform=ax.get_xaxis_transform())

# Spectrum:
fft_mean = np.mean(abs(fft_t2m), axis=1)
fft_std = np.std(abs(fft_t2m), axis=1)
ax.plot(freq[mask], fft_mean[mask])
ax.fill_between(freq[mask], (fft_mean - fft_std)[mask], (fft_mean + fft_std)[mask], alpha=0.5)

ax.set_xscale('log')
ax.set_yscale('log')
ax.set(xlabel="Frequency / Hz", ylabel="Magnitude", title="T2M")
plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.tight_layout()

# Day:
ax.axvline(1/(24*60*60), color="tab:grey", ls="dashed")
ax.text(1/(24*60*60), 0.99, 'day', color='tab:grey', ha='right', va='top', rotation=90, transform=ax.get_xaxis_transform())

# Month:
ax.axvline(1/(30*24*60*60), color="tab:grey", ls="dashed")
ax.text(1/(30*24*60*60), 0.99, 'month', color='tab:grey', ha='right', va='top', rotation=90, transform=ax.get_xaxis_transform())

# Season:
ax.axvline(4/(365*24*60*60), color="tab:grey", ls="dashed")
ax.text(4/(365*24*60*60), 0.99, 'season', color='tab:grey', ha='right', va='top', rotation=90, transform=ax.get_xaxis_transform())

# Year:
ax.axvline(1/(365*24*60*60), color="tab:grey", ls="dashed")
ax.text(1/(365*24*60*60), 0.99, 'year', color='tab:grey', ha='right', va='top', rotation=90, transform=ax.get_xaxis_transform())

# Spectrum:
fft_mean = np.mean(abs(fft_gpp), axis=1)
fft_std = np.std(abs(fft_gpp), axis=1)
ax.plot(freq[mask], fft_mean[mask])
ax.fill_between(freq[mask], (fft_mean - fft_std)[mask], (fft_mean + fft_std)[mask], alpha=0.5)

ax.set_xscale('log')
ax.set_yscale('log')
ax.set(xlabel="Frequency / Hz", ylabel="Magnitude", title="aco2gpp")
plt.show()